In [ ]:
from web3 import Web3
from web3.exceptions import ContractLogicError, Web3RPCError
import json
from datetime import datetime
import boto3
from botocore.exceptions import ClientError
from decimal import Decimal
import os 
import requests

In [13]:
os.getenv('blockchain_class_access_key')

In [3]:
def create_receipts_table(table_name):
    try:
        # Check if the table already exists
        table = dynamodb.Table(table_name)
        table.load()  # This will raise an error if the table does not exist
        print(f"Table '{table_name}' already exists.")
    except ClientError as e:
        # If the table doesn't exist, a ResourceNotFoundException will be raised
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            # Create the table if it does not exist
            table = dynamodb.create_table(
                TableName=table_name,
                KeySchema=[
                    {
                        'AttributeName': 'transaction_hash',
                        'KeyType': 'HASH'  # Partition key
                    }
                ],
                AttributeDefinitions=[
                    {
                        'AttributeName': 'transaction_hash',
                        'AttributeType': 'S'  # String type
                    }
                ],
                ProvisionedThroughput={
                    'ReadCapacityUnits': 5,
                    'WriteCapacityUnits': 5
                }
            )
            # Wait until the table exists
            table.meta.client.get_waiter('table_exists').wait(TableName=table_name)
            print(f"Table '{table_name}' created successfully.")
        else:
            # Handle other errors
            print("Error creating table:", e.response['Error']['Message'])
def create_sellers_table():
    dynamodb = boto3.resource('dynamodb', region_name='us-east-2')
    
    # Create the Sellers table
    table = dynamodb.create_table(
        TableName='Sellers',
        KeySchema=[
            {
                'AttributeName': 'seller_address',
                'KeyType': 'HASH'  # Partition key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'seller_address',
                'AttributeType': 'S'
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )
    
    # Wait until the table is created
    table.meta.client.get_waiter('table_exists').wait(TableName='Sellers')
    print("Sellers table created successfully.")

In [16]:
dynamodb = boto3.resource(
    'dynamodb',
    region_name='us-east-2',  # Update with your region
    aws_access_key_id=os.getenv('blockchain_class_access_key'),
    aws_secret_access_key=os.getenv('blockchain_class_secret_key')
)

In [17]:
class SellersManager:
    def __init__(self, table_name='Sellers'):
        self.dynamodb = boto3.resource(
            'dynamodb',
            region_name='us-east-2',
            aws_access_key_id=os.getenv('blockchain_class_access_key'),
            aws_secret_access_key=os.getenv('blockchain_class_secret_key')
        )
        self.table = self.dynamodb.Table(table_name)

    def insert_seller(self, seller_data):
        """
        Inserts a new seller record into the Sellers table.
        Expects seller_data to be a dictionary with 'seller_address' and 'seller_contract'.
        """
        try:
            response = self.table.put_item(
                Item=seller_data,
                ConditionExpression="attribute_not_exists(seller_address)"
            )
            print("Seller inserted successfully:", response)
        except ClientError as e:
            if e.response['Error']['Code'] == 'ConditionalCheckFailedException':
                print("Seller already exists.")
            else:
                print("Error inserting seller:", e.response['Error']['Message'])

    def seller_exists(self, seller_address):
        """
        Checks if a seller exists in the table.
        Returns True if the seller exists, False otherwise.
        """
        try:
            response = self.table.get_item(Key={'seller_address': seller_address})
            return 'Item' in response  # Returns True if item exists, False otherwise
        except ClientError as e:
            print("Error checking seller existence:", e.response['Error']['Message'])
            return False
    def get_all_sellers(self):
        """
        Retrieves all sellers with their associated contract addresses.
        Returns a list of dictionaries, each containing 'seller_address' and 'seller_contract_address'.
        """
        try:
            sellers = []
            response = self.table.scan(
                ProjectionExpression="seller_address, seller_contract_address"
            )
            
            # Append the first batch of items
            sellers.extend(response.get('Items', []))
            
            # Continue fetching if there are more items (pagination)
            while 'LastEvaluatedKey' in response:
                response = self.table.scan(
                    ProjectionExpression="seller_address, seller_contract_address",
                    ExclusiveStartKey=response['LastEvaluatedKey']
                )
                sellers.extend(response.get('Items', []))
                
            print(f"Retrieved {len(sellers)} sellers.")
            return sellers
        except ClientError as e:
            print(f"Error retrieving sellers: {e.response['Error']['Message']}")
            return []

In [18]:
class ReceiptManager:
    def __init__(self,table_name):
        self.dynamodb = boto3.resource(
            'dynamodb',
            region_name='us-east-2',  # Update with your region
            aws_access_key_id=os.getenv('blockchain_class_access_key'),
            aws_secret_access_key=os.getenv('blockchain_class_secret_key')
        )
        # create_receipts_table(table_name)
        self.table = self.dynamodb.Table(table_name)

    def insert_receipt(self, receipt_details):
        """Inserts a new receipt record in DynamoDB."""
        try:
            if 'amount' in receipt_details:
                receipt_details['amount'] = Decimal(str(receipt_details['amount']))
            
            # Insert the record, only if transaction_hash doesn't already exist
            response = self.table.put_item(
                Item=receipt_details
            )
            print("Data saved successfully:", response)
        except ClientError as e:
            print("Error saving data to DynamoDB:", e.response['Error']['Message'])

    def search_by_transaction_id(self, transaction_id):
        """Searches for a receipt by transaction ID (primary key)."""
        try:
            response = self.table.get_item(Key={'transaction_hash': transaction_id})
            return response.get('Item')
        except ClientError as e:
            print(f"Failed to retrieve receipt: {e.response['Error']['Message']}")
            return None

    def search_by_buyer_address(self, buyer_address, filter_by=None, sort_by=None, ascending=True):
        """Searches receipts by buyer address with optional filtering and sorting."""
        return self._search_by_attribute('buyer_address', buyer_address, filter_by, sort_by, ascending)

    def search_by_seller_address(self, seller_address, filter_by=None, sort_by=None, ascending=True):
        """Searches receipts by seller address with optional filtering and sorting."""
        return self._search_by_attribute('seller_contract_address', seller_address, filter_by, sort_by, ascending)

    def _search_by_attribute(self, attribute, value, filter_by=None, sort_by=None, ascending=True):
        """Internal method to search by a specific attribute (buyer or seller) with filtering and sorting."""
        try:
            # Query using the specified attribute
            response = self.table.scan(
                FilterExpression=boto3.dynamodb.conditions.Attr(attribute).eq(value)
            )
            items = response.get('Items', [])
            
            # Filter items if a filter criterion is provided
            if filter_by:
                if 'amount' in filter_by:
                    amount_filter = Decimal(str(filter_by['amount']))
                    items = [item for item in items if item['amount'] == amount_filter]
                if 'purchase_time' in filter_by:
                    items = [item for item in items if item['purchase_time'] == filter_by['purchase_time']]

            # Sort items if a sorting criterion is provided
            if sort_by:
                reverse_order = not ascending
                if sort_by == 'amount':
                    items.sort(key=lambda x: x['amount'], reverse=reverse_order)
                elif sort_by == 'purchase_time':
                    items.sort(key=lambda x: x['purchase_time'], reverse=reverse_order)
            
            return items
        except ClientError as e:
            print(f"Failed to search receipts: {e.response['Error']['Message']}")
            return None
    def get_all_transactions(self,max_number_of_pages = 5):
        """Retrieves all transactions from the DynamoDB table."""
        try:
            # Initialize an empty list to store all transactions
            transactions = []

            # Use the scan operation to retrieve all items
            response = self.table.scan()

            # Append the first batch of items to the list
            transactions.extend(response.get('Items', []))

            # Continue fetching if there are more items
            page = 1
            while 'LastEvaluatedKey' in response:
                if page>max_number_of_pages:
                    break
                response = self.table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
                transactions.extend(response.get('Items', []))
                page+=1
                
            print(f"Retrieved {len(transactions)} transactions.")
            return transactions
        except ClientError as e:
            print(f"Error retrieving transactions: {e.response['Error']['Message']}")
            return None
    def get_unique_sellers(self):
        """Retrieves all unique seller addresses with their associated contract addresses."""
        try:
            unique_sellers = set()
            sellers = []
            response = self.table.scan(
                ProjectionExpression="seller_address, seller_contract_address"
            )
            
            for item in response.get('Items', []):
                seller_info = (item['seller_address'], item['seller_contract_address'])
                if seller_info not in unique_sellers:
                    unique_sellers.add(seller_info)
                    sellers.append({'seller_address': item['seller_address'], 'seller_contract_address': item['seller_contract_address']})
                
            while 'LastEvaluatedKey' in response:
                response = self.table.scan(
                    ProjectionExpression="seller_address, seller_contract_address",
                    ExclusiveStartKey=response['LastEvaluatedKey']
                )
                for item in response.get('Items', []):
                    seller_info = (item['seller_address'], item['seller_contract_address'])
                    if seller_info not in unique_sellers:
                        unique_sellers.add(seller_info)
                        sellers.append({'seller_address': item['seller_address'], 'seller_contract_address': item['seller_contract_address']})

            print(f"Found {len(sellers)} unique sellers.")
            return sellers
        except ClientError as e:
            print(f"Error retrieving unique sellers: {e.response['Error']['Message']}")
            return []

    def get_unique_buyers(self):
        """Retrieves all unique buyer addresses."""
        try:
            unique_buyers = set()
            buyers = []
            response = self.table.scan(
                ProjectionExpression="buyer_address"
            )
            
            for item in response.get('Items', []):
                buyer_address = item['buyer_address']
                if buyer_address not in unique_buyers:
                    unique_buyers.add(buyer_address)
                    buyers.append(buyer_address)
                
            while 'LastEvaluatedKey' in response:
                response = self.table.scan(
                    ProjectionExpression="buyer_address",
                    ExclusiveStartKey=response['LastEvaluatedKey']
                )
                for item in response.get('Items', []):
                    buyer_address = item['buyer_address']
                    if buyer_address not in unique_buyers:
                        unique_buyers.add(buyer_address)
                        buyers.append(buyer_address)

            print(f"Found {len(buyers)} unique buyers.")
            return buyers
        except ClientError as e:
            print(f"Error retrieving unique buyers: {e.response['Error']['Message']}")
            return []

In [19]:
receipt_Dynamo_DB = ReceiptManager('Receipts')

In [20]:
receipt_Dynamo_DB.get_unique_buyers()

Error retrieving unique buyers: Requested resource not found


[]

In [9]:
receipt_Dynamo_DB.get_unique_sellers()

Error retrieving unique sellers: Requested resource not found


[]

In [102]:
ganache_url = "http://127.0.0.1:8545"  # Default Ganache URL
web3 = Web3(Web3.HTTPProvider(ganache_url))

In [103]:
assert web3.is_connected()

In [104]:
# contract_address = "0xb79b707E68b8e7c06A3da63f17658C108f23Fbd5"  # Replace with your deployed contract address
with open("build/contracts/ReceiptManager.json") as f:
    contract_json = json.load(f)
    contract_abi = contract_json["abi"]
    contract_bytecode = contract_json["bytecode"]

In [106]:
def issue_receipt(contract_address, seller_address, buyer_address, amount_eth):
    """Issues a receipt for the given buyer address and amount (in Ether) using a specific contract."""
    # Convert amount to Wei, since Ether is the base unit in web3.py
    amount_wei = web3.to_wei(amount_eth, 'ether')
    
    # Create a contract instance for the specific seller's contract address
    seller_contract = web3.eth.contract(address=contract_address, abi=contract_abi)
    
    # Send the transaction to the contract's issueReceipt function
    tx_hash = seller_contract.functions.issueReceipt(buyer_address).transact({
        'from': buyer_address,  # Pass in the seller's address from the API
        'value': amount_wei  # The amount to hold in escrow
    })
    
    # Wait for the transaction receipt to confirm
    tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)

    # Retrieve the actual timestamp from the block containing this transaction
    block = web3.eth.get_block(tx_receipt['blockNumber'])
    purchase_time = datetime.utcfromtimestamp(block['timestamp']).strftime('%Y-%m-%d %H:%M:%S')
    
    # Retrieve the ReceiptIssued event data from the transaction receipt
    receipt_event = seller_contract.events.ReceiptIssued().process_log(tx_receipt.logs[0])
    
    # Extract the receiptIndex from the event
    receipt_index = receipt_event['args']['receiptIndex']
    
    # Format the receipt object to return to the frontend
    receipt_details = {
        "buyer_address": buyer_address,
        "seller_address":seller_address,
        "seller_contract_address": contract_address,
        "amount": amount_eth,  # Return amount in Ether for readability
        "purchase_time":  purchase_time,
        "transaction_hash": tx_receipt['transactionHash'].hex(),
        "block_number": tx_receipt['blockNumber'],
        "status": "Success" if tx_receipt['status'] == 1 else "Failed",
        "receipt_index": receipt_index
    }
    return receipt_details


In [108]:
def decode_revert_message(error_data):
    """Decodes the revert reason from the error data."""
    # Standard error signature for revert reason
    if error_data.startswith("0x08c379a0"):
        try:
            # Decode the hex revert message
            revert_reason = Web3.toText(hexstr=error_data[10:]).strip()
            return revert_reason
        except Exception as e:
            return "Revert reason decoding failed"
    return "Unknown error"

def request_return(contract_address, buyer_address, receiptIndex):
    """Request a return for a specific receipt and capture revert reasons if it fails."""
    # Create a contract instance for the specific seller's contract address
    contract = web3.eth.contract(address=contract_address, abi=contract_abi)
    try:
        tx_hash = contract.functions.requestReturn(receiptIndex).transact({
            'from': buyer_address
        })
        
        # Wait for the transaction receipt
        tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
        
        # Return transaction details if successful
        return {
            "transaction_hash": tx_receipt['transactionHash'].hex(),
            "status": "Success" if tx_receipt['status'] == 1 else "Failed",
            "transaction_receipt": tx_receipt
        }
        
    except Web3RPCError as e:
        # Decode any unexpected errors during the actual transact call
        error_message = eval(e.args[0]).get('message', '')
        # error_message = decode_revert_message(error_data)
        return {
            "status": "Failed",
            "reason": error_message
        }


In [110]:
def release_funds(contract_address, buyer_address, receipt_index, seller_address):
    """Releases funds to the seller after the return window has expired."""
    contract = web3.eth.contract(address=contract_address, abi=contract_abi)
    try:
        tx_hash = contract.functions.releaseFunds(buyer_address, receipt_index).transact({
            'from': seller_address
        })
    
        # Wait for the transaction receipt
        tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
        
        return {
            "transaction_hash": tx_receipt['transactionHash'].hex(),
            "status": "Success" if tx_receipt['status'] == 1 else "Failed",
            "transaction_receipt": tx_receipt
        }
    except ContractLogicError as e:
        # Decode any unexpected errors during the actual transact call
        print(e.args[1])
        error_message = e.args[1].get('reason', '')
        # error_message = decode_revert_message(error_data)
        return {
            "status": "Failed",
            "reason": error_message
        }


In [111]:
def get_receipts(buyer_address):
    """Fetches all receipts for the given buyer address."""
    receipts = contract.functions.getReceipts(buyer_address).call()
    for i, receipt in enumerate(receipts):
        amount, time, refunded = receipt
        print(f"Receipt {i}: Amount - {web3.fromWei(amount, 'ether')} ETH, "
              f"Time - {time}, Refunded - {refunded}")

In [112]:
def deploy_new_contract(seller_account,return_window_days):
    """Deploy a new instance of the ReceiptManager contract and return the address."""
    ReceiptManager = web3.eth.contract(abi=contract_abi, bytecode=contract_bytecode)
    tx_hash = ReceiptManager.constructor(return_window_days).transact({'from': seller_account})
    tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    return tx_receipt.contractAddress

In [118]:
sellers_manager = SellersManager()

In [119]:
seller_address = web3.eth.accounts[1]
new_contract_seller_address = deploy_new_contract(seller_address,30)
buyer_address = web3.eth.accounts[2]

In [120]:
sellers_manager.insert_seller({'seller_address':seller_address,'seller_contract_address':new_contract_seller_address})

Seller inserted successfully: {'ResponseMetadata': {'RequestId': 'CVCUCL9BRU2S0PHELOF8VS8O77VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 13 Nov 2024 18:18:27 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'CVCUCL9BRU2S0PHELOF8VS8O77VV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}


In [121]:
sellers_manager.seller_exists(buyer_address)

False

In [122]:
all_sellers = sellers_manager.get_all_sellers()

Retrieved 6 sellers.


In [123]:
all_sellers

[{'seller_address': '0xFF746A9dD4EFD7b40D1567672DCBF88bEF29D828',
  'seller_contract_address': '0x33CB038CFa037f106032E7073cb7dD15BA9d74A3'},
 {'seller_address': '0x1BEC335E19c7e6685128149A61FDB40C4683C4F5',
  'seller_contract_address': '0xbaE5A4E5732E0b5bA773f94F875C73B5c5c67B8B'},
 {'seller_address': '0xBCC0684109EB8CDcF248C91cfFA844Cfe2D38c02',
  'seller_contract_address': '0x46872b93700007C6A962635aDd54754fFbbdebd3'},
 {'seller_address': '0x4484de6868944DD66556aB81cDd9b8BDd7cAd906',
  'seller_contract_address': '0x493825b0A57a39aBDe5fbE1D6Ae7504d65191aD7'},
 {'seller_address': '0x2f62FE92c12a3dA8009fdfe199E0D22f47A865Fa',
  'seller_contract_address': '0x1FAFF8Bf1853030D34262985E98FfC016f910A4f'},
 {'seller_address': '0x5F7CDd84e4aF0080B19749f8F345Bf2303CA6De0',
  'seller_contract_address': '0x5460519EE6fF5f40696c2C6902b5Ac4d81514363'}]

In [124]:
receipt_event = issue_receipt(new_contract_seller_address, seller_address, buyer_address, 0.1)  # Issue receipt for 0.1 Ether

/var/folders/fy/mj05n9c54hqbvbww98k7rnw00000gn/T/ipykernel_2393/2186493913.py:20: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  purchase_time = datetime.utcfromtimestamp(block['timestamp']).strftime('%Y-%m-%d %H:%M:%S')


In [125]:
receipt_event['item'] = 'Pokemon Cards'

In [126]:
receipt_event

{'buyer_address': '0x5204De0498cED1195fAf4c66d52011DA61F04a11',
 'seller_address': '0xFF746A9dD4EFD7b40D1567672DCBF88bEF29D828',
 'seller_contract_address': '0x33CB038CFa037f106032E7073cb7dD15BA9d74A3',
 'amount': 0.1,
 'purchase_time': '2024-11-13 18:19:02',
 'transaction_hash': 'bffd311e10de6216ea00ddf69d2f2e9fab373a40da02b09f33973854a8e4064d',
 'block_number': 3,
 'status': 'Success',
 'receipt_index': 0,
 'item': 'Pokemon Cards'}

In [127]:
receipt_Dynamo_DB.insert_receipt(receipt_event)

Data saved successfully: {'ResponseMetadata': {'RequestId': '5PL0KJ1CGRTJSOA7FVBEKHCHE7VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 13 Nov 2024 18:20:02 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': '5PL0KJ1CGRTJSOA7FVBEKHCHE7VV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}


In [128]:
request_return(new_contract_seller_address, buyer_address, receipt_event['receipt_index'])

{'transaction_hash': 'd21a2415a1c3ebdb42f5cb75f0480eab16175ab02766c18c770e20d8837eb77f',
 'status': 'Success',
 'transaction_receipt': AttributeDict({'transactionHash': HexBytes('0xd21a2415a1c3ebdb42f5cb75f0480eab16175ab02766c18c770e20d8837eb77f'),
  'transactionIndex': 0,
  'blockNumber': 4,
  'blockHash': HexBytes('0xa80ed0cbafb229bb17466cfef98d15d0a20edcbfb9739744d547f4f296f7fbb9'),
  'from': '0x5204De0498cED1195fAf4c66d52011DA61F04a11',
  'to': '0x33CB038CFa037f106032E7073cb7dD15BA9d74A3',
  'cumulativeGasUsed': 61583,
  'gasUsed': 61583,
  'contractAddress': None,
  'logs': [AttributeDict({'address': '0x33CB038CFa037f106032E7073cb7dD15BA9d74A3',
    'blockHash': HexBytes('0xa80ed0cbafb229bb17466cfef98d15d0a20edcbfb9739744d547f4f296f7fbb9'),
    'blockNumber': 4,
    'data': HexBytes('0x000000000000000000000000000000000000000000000000016345785d8a0000'),
    'logIndex': 0,
    'removed': False,
    'topics': [HexBytes('0xa171b6942063c6f2800ce40a780edce37baa2b618571b11eedd1e69e626e7d

In [129]:
release_funds(new_contract_seller_address, buyer_address,0, seller_address)

{'hash': None, 'programCounter': 2122, 'result': '0x08c379a000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000015526566756e6420616c7265616479206973737565640000000000000000000000', 'reason': 'Refund already issued', 'message': 'revert'}


{'status': 'Failed', 'reason': 'Refund already issued'}

In [130]:
balance_wei = web3.eth.get_balance(new_contract_seller_address)

# Convert the balance to Ether for easier readability
balance_eth = web3.from_wei(balance_wei, 'ether')

print(f"Contract Balance: {balance_eth} ETH")

Contract Balance: 0 ETH


In [132]:
if web3.is_connected():
    # Retrieve all accounts from Ganache
    accounts = web3.eth.accounts

    # Print each account's address and balance in Ether
    for account in accounts:
        balance_wei = web3.eth.get_balance(account)
        balance_eth = web3.from_wei(balance_wei, 'ether')
        print(f"Account: {account}, Balance: {balance_eth} ETH")
else:
    print("Unable to connect to Ganache.")

Account: 0xE1c5A7F0ABEd736915ecB9aca7E04f7F2A842A8b, Balance: 99.996357602125 ETH
Account: 0xFF746A9dD4EFD7b40D1567672DCBF88bEF29D828, Balance: 99.998906261901443908 ETH
Account: 0x5204De0498cED1195fAf4c66d52011DA61F04a11, Balance: 99.999862539604065224 ETH
Account: 0x08298c261175cdBae5CcED3E5DB216e681A2c774, Balance: 100 ETH
Account: 0x59e288D36CA7Ed764174Fb49c33F2017A8B94526, Balance: 100 ETH
Account: 0xbAa1C3Ed844a92cCb20D4985590Cb1fC9e1BfF76, Balance: 100 ETH
Account: 0xEe23effB1AfBF7DEC413872D8fe0D946f6ad8a28, Balance: 100 ETH
Account: 0x04457496C7Baf1cF004A81417b28F2314AABDcA9, Balance: 100 ETH
Account: 0x5797955D09731e6c1455b1053E360B94C1EE1D72, Balance: 100 ETH
Account: 0x1b3cb5108da53921fA45B60bcDa5C0e34210b832, Balance: 100 ETH


In [133]:
all_transactions = receipt_Dynamo_DB.get_all_transactions()

Retrieved 5 transactions.


In [134]:
all_transactions

[{'return_time': Decimal('1731426287.7691590785980224609375'),
  'item_name': 'Pokemon Cards',
  'seller_address': '0x4484de6868944DD66556aB81cDd9b8BDd7cAd906',
  'status': 'Returned',
  'amount': Decimal('0.2'),
  'block_number': Decimal('3'),
  'seller_contract_address': '0x493825b0A57a39aBDe5fbE1D6Ae7504d65191aD7',
  'transaction_hash': '1a623bd66bac4f29b9330032c3e703a4745cf79728cf87cd02a7a55e83bf48f0',
  'receipt_index': Decimal('0'),
  'buyer_address': '0x1BEC335E19c7e6685128149A61FDB40C4683C4F5',
  'purchase_time': '2024-11-12 15:41:06'},
 {'return_time': Decimal('1731467705.3704888820648193359375'),
  'item_name': 'Pokemon Cards',
  'seller_address': '0x2f62FE92c12a3dA8009fdfe199E0D22f47A865Fa',
  'status': 'Returned',
  'amount': Decimal('0.2'),
  'block_number': Decimal('5'),
  'seller_contract_address': '0x1FAFF8Bf1853030D34262985E98FfC016f910A4f',
  'transaction_hash': 'da507f0edaca51b09859cbcd0639fe921adf2227a541fd822296aa35ea5e5e94',
  'receipt_index': Decimal('0'),
  'buy

In [153]:
requests.get('http://0.0.0.0:8010/reset_tables')

<Response [200]>

In [154]:
seller_address = web3.eth.accounts[0]
buyer_address = web3.eth.accounts[1]

In [155]:
r = requests.post('http://0.0.0.0:8010/create_seller_contract',json = {'seller_account_address':seller_address,'return_window_days':30})

In [156]:
r.json()

{'seller_address': '0x4484de6868944DD66556aB81cDd9b8BDd7cAd906',
 'contract_address': '0x493825b0A57a39aBDe5fbE1D6Ae7504d65191aD7',
 'return_window_days': 30}

In [157]:
r = requests.get('http://0.0.0.0:8010/get_sellers_w_contracts')
r.json()

{'0x4484de6868944DD66556aB81cDd9b8BDd7cAd906': {'return_window_days': 30,
  'seller_address': '0x4484de6868944DD66556aB81cDd9b8BDd7cAd906',
  'seller_contract_address': '0x493825b0A57a39aBDe5fbE1D6Ae7504d65191aD7'}}

In [158]:
r = requests.post('http://0.0.0.0:8010/issue_receipt',
                  json = {'seller_address':seller_address,'buyer_address':buyer_address,'amount_eth':.2,'item_name':'Pokemon Cards'})
r.json()

{'success': True,
 'receipt_details': {'buyer_address': '0x1BEC335E19c7e6685128149A61FDB40C4683C4F5',
  'seller_address': '0x4484de6868944DD66556aB81cDd9b8BDd7cAd906',
  'seller_contract_address': '0x493825b0A57a39aBDe5fbE1D6Ae7504d65191aD7',
  'amount': 0.2,
  'purchase_time': '2024-11-12 15:41:06',
  'transaction_hash': '1a623bd66bac4f29b9330032c3e703a4745cf79728cf87cd02a7a55e83bf48f0',
  'block_number': 3,
  'status': 'Active',
  'receipt_index': 0,
  'item_name': 'Pokemon Cards'}}

In [182]:
r = requests.get('http://0.0.0.0:8010/get_all_accounts_in_network')
r.json()

{'all_accounts': [{'account_index': 0,
   'account_address': '0x715303E222379d51A33050a03d0ca9F74De79430',
   'balance': 999.99625274425},
  {'account_index': 1,
   'account_address': '0x088e980DcF726b68a93DDF8A533840C0036E97e2',
   'balance': 1000},
  {'account_index': 2,
   'account_address': '0xdc030AbcE41860dACE1bDB6A2A481E8d69deAd09',
   'balance': 1000},
  {'account_index': 3,
   'account_address': '0xf8434966f5dD0a69DA1a774d2250b0732cCe3445',
   'balance': 1000},
  {'account_index': 4,
   'account_address': '0x98eb44f5b46f3705CF352D70B368d0C16EbecF14',
   'balance': 1000},
  {'account_index': 5,
   'account_address': '0x65c6204f7c86A7b5DC8CDBEC9556991Dc14398C4',
   'balance': 1000},
  {'account_index': 6,
   'account_address': '0x29122eeB40E4d8A06F0B871bBf00F21CFB4b16B1',
   'balance': 1000},
  {'account_index': 7,
   'account_address': '0x7dEA308da586a152740E80AB023A7358d381f63b',
   'balance': 1000},
  {'account_index': 8,
   'account_address': '0x5DA8c48993C35A9e1d00BEC33Be16

In [160]:
r = requests.post('http://0.0.0.0:8010/get_buyer_receipts',
                  json = {'buyer_address':buyer_address})
buyer_receipts = r.json()['all_receipts']
buyer_receipts

[{'item_name': 'Pokemon Cards',
  'seller_address': '0x4484de6868944DD66556aB81cDd9b8BDd7cAd906',
  'status': 'Active',
  'amount': 0.2,
  'block_number': 3,
  'seller_contract_address': '0x493825b0A57a39aBDe5fbE1D6Ae7504d65191aD7',
  'transaction_hash': '1a623bd66bac4f29b9330032c3e703a4745cf79728cf87cd02a7a55e83bf48f0',
  'receipt_index': 0,
  'buyer_address': '0x1BEC335E19c7e6685128149A61FDB40C4683C4F5',
  'purchase_time': '2024-11-12 15:41:06'}]

In [163]:
current_receipt = buyer_receipts[0]
r = requests.post('http://0.0.0.0:8010/request_return',
                  json = {'transaction_hash':current_receipt['transaction_hash']})
r.json()

{'success': True,
 'return_request_details': "{'transaction_hash': '2c4c34513664968ef39e2b8f46c12480559f1606ecf37f40fd1226a6296bf179', 'status': 'Success', 'transaction_receipt': AttributeDict({'transactionHash': HexBytes('0x2c4c34513664968ef39e2b8f46c12480559f1606ecf37f40fd1226a6296bf179'), 'transactionIndex': 0, 'blockNumber': 4, 'blockHash': HexBytes('0x9950aae6cc91035732d29e373d4490608e98564221a6c5be42046590456ba617'), 'from': '0x1BEC335E19c7e6685128149A61FDB40C4683C4F5', 'to': '0x493825b0A57a39aBDe5fbE1D6Ae7504d65191aD7', 'cumulativeGasUsed': 61583, 'gasUsed': 61583, 'contractAddress': None, 'logs': [AttributeDict({'address': '0x493825b0A57a39aBDe5fbE1D6Ae7504d65191aD7', 'blockHash': HexBytes('0x9950aae6cc91035732d29e373d4490608e98564221a6c5be42046590456ba617'), 'blockNumber': 4, 'data': HexBytes('0x00000000000000000000000000000000000000000000000002c68af0bb140000'), 'logIndex': 0, 'removed': False, 'topics': [HexBytes('0xa171b6942063c6f2800ce40a780edce37baa2b618571b11eedd1e69e626e

In [164]:
r = requests.post('http://0.0.0.0:8010/get_buyer_receipts',
                  json = {'buyer_address':buyer_address})
buyer_receipts = r.json()['all_receipts']
buyer_receipts

[{'return_time': 1731426287.769159,
  'item_name': 'Pokemon Cards',
  'seller_address': '0x4484de6868944DD66556aB81cDd9b8BDd7cAd906',
  'status': 'Returned',
  'amount': 0.2,
  'block_number': 3,
  'seller_contract_address': '0x493825b0A57a39aBDe5fbE1D6Ae7504d65191aD7',
  'transaction_hash': '1a623bd66bac4f29b9330032c3e703a4745cf79728cf87cd02a7a55e83bf48f0',
  'receipt_index': 0,
  'buyer_address': '0x1BEC335E19c7e6685128149A61FDB40C4683C4F5',
  'purchase_time': '2024-11-12 15:41:06'}]

In [165]:
current_receipt = buyer_receipts[0]
r = requests.post('http://0.0.0.0:8010/request_return',
                  json = {'transaction_hash':current_receipt['transaction_hash']})
r.json()

{'detail': '500: Refund already issued'}

In [166]:
r = requests.post('http://0.0.0.0:8010/get_seller_receipts',
                  json = {'seller_address':seller_address})
seller_receipts = r.json()
seller_receipts

{'success': True,
 'all_receipts': [{'return_time': 1731426287.769159,
   'item_name': 'Pokemon Cards',
   'seller_address': '0x4484de6868944DD66556aB81cDd9b8BDd7cAd906',
   'status': 'Returned',
   'amount': 0.2,
   'block_number': 3,
   'seller_contract_address': '0x493825b0A57a39aBDe5fbE1D6Ae7504d65191aD7',
   'transaction_hash': '1a623bd66bac4f29b9330032c3e703a4745cf79728cf87cd02a7a55e83bf48f0',
   'receipt_index': 0,
   'buyer_address': '0x1BEC335E19c7e6685128149A61FDB40C4683C4F5',
   'purchase_time': '2024-11-12 15:41:06'}]}

In [168]:
seller_receipts = buyer_receipts[0]
r = requests.post('http://0.0.0.0:8010/release_funds',
                  json = {'transaction_hash':current_receipt['transaction_hash']})
r.json()

{'detail': '500: Refund already issued'}

In [169]:
seller_address = web3.eth.accounts[1]
buyer_address = web3.eth.accounts[2]

In [170]:
r = requests.post('http://0.0.0.0:8010/create_seller_contract',json = {'seller_account_address':seller_address,'return_window_days':0})

In [171]:
r = requests.get('http://0.0.0.0:8010/get_sellers_w_contracts')
r.json()

{'0x1BEC335E19c7e6685128149A61FDB40C4683C4F5': {'return_window_days': 0,
  'seller_address': '0x1BEC335E19c7e6685128149A61FDB40C4683C4F5',
  'seller_contract_address': '0xbaE5A4E5732E0b5bA773f94F875C73B5c5c67B8B'},
 '0x4484de6868944DD66556aB81cDd9b8BDd7cAd906': {'return_window_days': 30,
  'seller_address': '0x4484de6868944DD66556aB81cDd9b8BDd7cAd906',
  'seller_contract_address': '0x493825b0A57a39aBDe5fbE1D6Ae7504d65191aD7'}}

In [173]:
r = requests.post('http://0.0.0.0:8010/issue_receipt',
                  json = {'seller_address':seller_address,'buyer_address':buyer_address,'amount_eth':.2,'item_name':'One Piece Cards'})
r.json()

{'success': True,
 'receipt_details': {'buyer_address': '0x6C766E67fF42864F3EEeFEcED19119F993876707',
  'seller_address': '0x1BEC335E19c7e6685128149A61FDB40C4683C4F5',
  'seller_contract_address': '0xbaE5A4E5732E0b5bA773f94F875C73B5c5c67B8B',
  'amount': 0.2,
  'purchase_time': '2024-11-12 15:51:41',
  'transaction_hash': 'b320e73840b663433e72f079bf736c84baf67ed4cfab046ad9901487902a68fb',
  'block_number': 6,
  'status': 'Active',
  'receipt_index': 0,
  'item_name': 'One Piece Cards'}}

In [174]:
r = requests.post('http://0.0.0.0:8010/get_buyer_receipts',
                  json = {'buyer_address':buyer_address})
buyer_receipts = r.json()['all_receipts']
buyer_receipts

[{'item_name': 'One Piece Cards',
  'seller_address': '0x1BEC335E19c7e6685128149A61FDB40C4683C4F5',
  'status': 'Active',
  'amount': 0.2,
  'block_number': 6,
  'seller_contract_address': '0xbaE5A4E5732E0b5bA773f94F875C73B5c5c67B8B',
  'transaction_hash': 'b320e73840b663433e72f079bf736c84baf67ed4cfab046ad9901487902a68fb',
  'receipt_index': 0,
  'buyer_address': '0x6C766E67fF42864F3EEeFEcED19119F993876707',
  'purchase_time': '2024-11-12 15:51:41'}]

In [176]:
current_receipt = buyer_receipts[0]
r = requests.post('http://0.0.0.0:8010/request_return',
                  json = {'transaction_hash':current_receipt['transaction_hash']})
r.json()

{'detail': '500: Return window has closed'}

In [177]:
r = requests.post('http://0.0.0.0:8010/get_seller_receipts',
                  json = {'seller_address':seller_address})
seller_receipts = r.json()
seller_receipts

{'success': True,
 'all_receipts': [{'item_name': 'One Piece Cards',
   'seller_address': '0x1BEC335E19c7e6685128149A61FDB40C4683C4F5',
   'status': 'Active',
   'amount': 0.2,
   'block_number': 6,
   'seller_contract_address': '0xbaE5A4E5732E0b5bA773f94F875C73B5c5c67B8B',
   'transaction_hash': 'b320e73840b663433e72f079bf736c84baf67ed4cfab046ad9901487902a68fb',
   'receipt_index': 0,
   'buyer_address': '0x6C766E67fF42864F3EEeFEcED19119F993876707',
   'purchase_time': '2024-11-12 15:51:41'}]}

In [178]:
seller_receipts = buyer_receipts[0]
r = requests.post('http://0.0.0.0:8010/release_funds',
                  json = {'transaction_hash':current_receipt['transaction_hash']})
r.json()

{'success': True,
 'release_return_details': "{'transaction_hash': '2fd0c902bc9c81b4c0f1d0c253bdc7500c9b22537c53f7bdcd6c26f8997e9eea', 'status': 'Success', 'transaction_receipt': AttributeDict({'transactionHash': HexBytes('0x2fd0c902bc9c81b4c0f1d0c253bdc7500c9b22537c53f7bdcd6c26f8997e9eea'), 'transactionIndex': 0, 'blockNumber': 8, 'blockHash': HexBytes('0x40d9d754e21f81f48e5c8c030c8ad2a3b6973829fc986d9a70f626bebeb2cac5'), 'from': '0x1BEC335E19c7e6685128149A61FDB40C4683C4F5', 'to': '0xbaE5A4E5732E0b5bA773f94F875C73B5c5c67B8B', 'cumulativeGasUsed': 65156, 'gasUsed': 65156, 'contractAddress': None, 'logs': [AttributeDict({'address': '0xbaE5A4E5732E0b5bA773f94F875C73B5c5c67B8B', 'blockHash': HexBytes('0x40d9d754e21f81f48e5c8c030c8ad2a3b6973829fc986d9a70f626bebeb2cac5'), 'blockNumber': 8, 'data': HexBytes('0x00000000000000000000000000000000000000000000000002c68af0bb1400000000000000000000000000000000000000000000000000000000000000000000'), 'logIndex': 0, 'removed': False, 'topics': [HexBytes

In [179]:
r = requests.post('http://0.0.0.0:8010/get_seller_receipts',
                  json = {'seller_address':seller_address})
seller_receipts = r.json()
seller_receipts

{'success': True,
 'all_receipts': [{'item_name': 'One Piece Cards',
   'seller_address': '0x1BEC335E19c7e6685128149A61FDB40C4683C4F5',
   'status': 'Funds Released to Seller',
   'amount': 0.2,
   'block_number': 6,
   'seller_contract_address': '0xbaE5A4E5732E0b5bA773f94F875C73B5c5c67B8B',
   'transaction_hash': 'b320e73840b663433e72f079bf736c84baf67ed4cfab046ad9901487902a68fb',
   'funds_release_time': 1731426782.015963,
   'receipt_index': 0,
   'buyer_address': '0x6C766E67fF42864F3EEeFEcED19119F993876707',
   'purchase_time': '2024-11-12 15:51:41'}]}

In [180]:
seller_receipts = buyer_receipts[0]
r = requests.post('http://0.0.0.0:8010/release_funds',
                  json = {'transaction_hash':current_receipt['transaction_hash']})
r.json()

{'detail': '500: Funds already released'}